# Sheet Manager Prototype

This notebook prototypes the `SheetManager` class for managing collections of puzzle sheets.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from collections.abc import Callable
from pathlib import Path
from typing import Dict
from typing import List
from typing import Optional

from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet
# from snap_fit.puzzle.sheet_aruco import SheetAruco
# # Optional, if we want to test real loading

In [ ]:
class SheetManager:
    def __init__(self):
        self.sheets: dict[str, Sheet] = {}

    def add_sheet(self, sheet: Sheet, sheet_id: str) -> None:
        """Add a single sheet to the manager with a specific ID."""
        if sheet_id in self.sheets:
            lg.warning(f"Overwriting sheet with ID: {sheet_id}")
        self.sheets[sheet_id] = sheet
        lg.info(f"Added sheet: {sheet_id}")

    def add_sheets(
        self,
        folder_path: Path,
        pattern: str = "*",
        loader_func: Callable[[Path], Sheet] | None = None,
    ) -> None:
        """Glob a folder for files matching the pattern.

        Args:
            folder_path: Root directory to search.
            pattern: Glob pattern (e.g., "*.jpg", "*.json").
            loader_func: Function to convert a file Path into a Sheet object.
                         If None, assumes files are pickled Sheets or similar (TBD).

        Side Effects:
            - Generates an ID for each sheet (e.g., relative path from folder_path).
            - Populates self.sheets.
        """
        folder = Path(folder_path)
        if not folder.exists():
            lg.error(f"Folder not found: {folder}")
            return

        files = list(folder.glob(pattern))
        lg.info(f"Found {len(files)} files matching '{pattern}' in {folder}")

        for file_path in files:
            # Generate ID: relative path from the search folder
            # This ensures uniqueness within the context of this load
            sheet_id = str(file_path.relative_to(folder))

            if loader_func:
                sheet = loader_func(file_path)
                self.add_sheet(sheet, sheet_id)
            else:
                lg.warning(f"No loader_func provided, skipping {file_path}")
                # TODO: Implement default loading logic if applicable

    def get_sheet(self, sheet_id: str) -> Sheet | None:
        """Retrieve a sheet by its ID."""
        return self.sheets.get(sheet_id)

    def get_sheets_ls(self) -> list[Sheet]:
        """Return a list of all managed sheets."""
        return list(self.sheets.values())

    def get_pieces_ls(self) -> list[Piece]:
        """Return a flat list of all pieces across all sheets."""
        # Assuming Sheet has a 'pieces' attribute which is a list of Piece objects
        # We need to verify Sheet structure. Based on previous read, Sheet is a class.
        # Let's assume for now it has a .pieces attribute or we need to inspect it.
        # Looking at Sheet definition again might be good,
        # but for prototype we can mock it.
        all_pieces = []
        for sheet in self.sheets.values():
            if hasattr(sheet, "pieces"):
                all_pieces.extend(sheet.pieces)
            else:
                lg.warning(f"Sheet {sheet} has no 'pieces' attribute")
        return all_pieces

In [ ]:
# Mocking for Prototype Validation


class MockPiece(Piece):
    def __init__(self, id):
        self.id = id

    def __repr__(self):
        return f"Piece({self.id})"


class MockSheet(Sheet):
    def __init__(self, name, num_pieces):
        self.name = name
        self.pieces = [MockPiece(f"{name}_p{i}") for i in range(num_pieces)]

    def __repr__(self):
        return f"Sheet({self.name}, {len(self.pieces)} pieces)"


def mock_loader(path: Path) -> Sheet:
    # Simulate loading a sheet from a file
    # In reality, this would parse the file or image
    return MockSheet(path.stem, num_pieces=5)


# Create dummy files for testing glob
test_dir = Path("test_data")
test_dir.mkdir(exist_ok=True, parents=True)
lg.debug(f"Created test directory at {test_dir.absolute()}")
(test_dir / "sheet1.txt").touch()
(test_dir / "sheet2.txt").touch()
(test_dir / "ignore.log").touch()

lg.info("Created mock data and classes")

In [ ]:
# Usage Example

manager = SheetManager()

# Batch load using the mock loader
manager.add_sheets(folder_path=test_dir, pattern="*.txt", loader_func=mock_loader)

# Verify Sheets
sheets = manager.get_sheets_ls()
print(f"Managed Sheets: {sheets}")

# Verify Pieces
pieces = manager.get_pieces_ls()
print(f"Total Pieces: {len(pieces)}")
print(f"Sample Pieces: {pieces[:3]}")

# Verify Get Sheet
sheet1 = manager.get_sheet("sheet1.txt")
print(f"Retrieved Sheet1: {sheet1}")

# Cleanup
import shutil

if test_dir.exists():
    shutil.rmtree(test_dir)
    lg.info("Cleaned up test data")